#Intro

In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.core import DoFn

#Code

`beam.CombineFn` to klasa w Apache Beam używana do definiowania niestandardowych operacji agregujących, które łączą wiele elementów w pojedynczą wartość. Jest szczególnie przydatna, gdy domyślne funkcje agregacji (np. `beam.CombinePerKey(sum)`) są niewystarczające i wymagane jest bardziej zaawansowane lub niestandardowe przetwarzanie.

In [4]:
class AverageFn(beam.CombineFn):
    # beam.CombineFn składa się z czterech kluczowych metod, które definiują różne etapy agregacji:

    # 1. create_accumulator()
    # • Tworzy pusty akumulator, który będzie używany do zbierania danych.
    def create_accumulator(self):
        return (0.0, 0)   # initialize (sum, count)

    # 2. add_input()
    # • Dodaje pojedynczy element do akumulatora.
    def add_input(self, sum_count, input):
        (sum, count) = sum_count
        return sum + input, count + 1

    # 3. merge_accumulators()
    # • Łączy wiele akumulatorów w jeden.
    def merge_accumulators(self, accumulators):
        ind_sums, ind_counts = zip(*accumulators)  # zip - [(27, 3), (39, 3), (18, 2)]  -->   [(27,39,18), (3,3,2)]
        return sum(ind_sums), sum(ind_counts)      # (84,8)

    # 4. extract_output()
    # • Wyodrębnia końcową wartość wyjściową z akumulatora.
    def extract_output(self, sum_count):
        (sum, count) = sum_count
        return sum / count if count else float('NaN')

# Inicjalizacja potoku
p = beam.Pipeline()

# Przykład użycia CombineGlobally
small_sum = (
    p
    | beam.Create([15, 5, 7, 7, 9, 23, 13, 5])  # Tworzenie danych wejściowych
    | "Combine Globally" >> beam.CombineGlobally(AverageFn())  # Użycie CombineGlobally z AverageFn
    | 'Write results' >> beam.io.WriteToText('data/combine')  # Zapis wyników do pliku
)

# Uruchomienie potoku
p.run()

In [5]:
!{'head -n 20 data/combine-00000-of-00001'}

10.5


In [ ]:

# {Średnia} = {Suma elementów} / {Liczba elementów} = {84} / {8} = 10.5


In [ ]:
class AverageFn(beam.CombineFn):
    # beam.CombineFn składa się z czterech kluczowych metod, które definiują różne etapy agregacji:

    # 1. create_accumulator()
    # • Tworzy pusty akumulator, który będzie używany do zbierania danych.
    # • W tym przypadku akumulator to krotka (sum, count), gdzie:
    #   - sum: przechowuje sumę wszystkich elementów
    #   - count: przechowuje liczbę elementów
    def create_accumulator(self):
        return (0.0, 0)  # Inicjalizacja akumulatora jako (0.0, 0)

    # 2. add_input()
    # • Dodaje pojedynczy element do akumulatora.
    # • Aktualizuje akumulator przez dodanie nowego elementu do sumy
    #   i zwiększenie licznika elementów o 1.
    def add_input(self, sum_count, input):
        (sum, count) = sum_count  # Rozpakowanie bieżącego akumulatora
        return sum + input, count + 1  # Zaktualizowany akumulator z nową sumą i licznikiem

    # 3. merge_accumulators()
    # • Łączy wiele akumulatorów w jeden.
    # • Używane w systemach rozproszonych, gdzie częściowe akumulatory muszą być scalane.
    def merge_accumulators(self, accumulators):
        # Rozpakowanie listy akumulatorów na osobne sumy i liczniki
        ind_sums, ind_counts = zip(*accumulators)
        # Zwrócenie scalonego akumulatora jako sumy wszystkich sum i sumy liczników
        return sum(ind_sums), sum(ind_counts)

    # 4. extract_output()
    # • Wyodrębnia końcową wartość wyjściową z akumulatora.
    # • Na końcu oblicza średnią jako suma / licznik.
    # • Jeżeli licznik jest równy 0, zwracana jest wartość NaN, aby uniknąć błędu dzielenia przez 0.
    def extract_output(self, sum_count):
        (sum, count) = sum_count  # Rozpakowanie końcowego akumulatora
        return sum / count if count else float('NaN')  # Obliczenie średniej lub zwrócenie NaN

# Inicjalizacja potoku
p = beam.Pipeline()

# Przykład użycia CombineGlobally
small_sum = (
    p
    | beam.Create([15, 5, 7, 7, 9, 23, 13, 5])  # Tworzenie danych wejściowych
    # • Użycie transformacji CombineGlobally z AverageFn
    # • Dane wejściowe są przekazywane do CombineFn, który oblicza średnią
    | "Combine Globally" >> beam.CombineGlobally(AverageFn())
    # Zapisanie wyniku do pliku tekstowego o nazwie 'data/combine'
    | 'Write results' >> beam.io.WriteToText('data/combine')
)

# Uruchomienie potoku
p.run()

In [10]:
class AverageFn(beam.CombineFn):
    def create_accumulator(self):
        accumulator = (0.0, 0)
        print(accumulator) # (0.0, 0)
        return accumulator

    def add_input(self, sum_count, input):
        (sum, count) = sum_count
        updated_accumulator = (sum + input, count + 1)
        print(updated_accumulator)
        # (15.0, 1)
        # (20.0, 2)

        return updated_accumulator

    def merge_accumulators(self, accumulators):
        print(accumulators) # [(84.0, 8)]

        ind_sums, ind_counts = zip(*accumulators)
        merged_accumulator = (sum(ind_sums), sum(ind_counts))
        print(merged_accumulator) # (84.0, 8)
        return merged_accumulator

    def extract_output(self, sum_count):
        (sum, count) = sum_count
        result = sum / count if count else float('NaN')
        print(f"{sum}, {count}, {result}") # 84.0, 8, 10.5
        return result

# Inicjalizacja potoku
p = beam.Pipeline()

# Przykład użycia CombineGlobally
small_sum = (
    p
    | beam.Create([15, 5, 7, 7, 9, 23, 13, 5])  # Tworzenie danych wejściowych
    | "Combine Globally" >> beam.CombineGlobally(AverageFn())  # Użycie CombineGlobally z AverageFn
    | 'Write results' >> beam.io.WriteToText('data/combine')  # Zapis wyników do pliku
)

# Uruchomienie potoku
p.run()

(0.0, 0)
(15.0, 1)
(20.0, 2)
(27.0, 3)
(34.0, 4)
(43.0, 5)
(66.0, 6)
(79.0, 7)
(84.0, 8)
[(84.0, 8)]
(84.0, 8)
84.0, 8, 10.5


Oblicza różnicę między maksymalną a minimalną wartością (max_value - min_value).

In [11]:
import apache_beam as beam

class MaxDifferenceFn(beam.CombineFn):
    # Tworzymy akumulator przechowujący minimum i maksimum
    def create_accumulator(self):
        accumulator = (float('inf'), float('-inf'))  # (min_value, max_value)
        print(f"create_accumulator: {accumulator}")
        # create_accumulator: (inf, -inf)
        return accumulator

    # Aktualizujemy akumulator o wartość elementu wejściowego
    def add_input(self, min_max, input):
        (current_min, current_max) = min_max
        updated_accumulator = (min(current_min, input), max(current_max, input))
        print(f"add_input - input: {input}, updated_accumulator: {updated_accumulator}")
        # add_input - input: 15, updated_accumulator: (15, 15)
        # add_input - input: 5, updated_accumulator: (5, 15)
        # add_input - input: 7, updated_accumulator: (5, 15)
        # add_input - input: 7, updated_accumulator: (5, 15)
        # add_input - input: 9, updated_accumulator: (5, 15)
        # add_input - input: 23, updated_accumulator: (5, 23)
        # add_input - input: 13, updated_accumulator: (5, 23)
        # add_input - input: 5, updated_accumulator: (5, 23)
        return updated_accumulator

    # Scalanie akumulatorów (np. przy obliczeniach rozproszonych)
    def merge_accumulators(self, accumulators):
        print(f"merge_accumulators - incoming accumulators: {accumulators}")
        # merge_accumulators - incoming accumulators: [(5, 23)]
        min_values, max_values = zip(*accumulators)
        merged_accumulator = (min(min_values), max(max_values))
        print(f"merge_accumulators - merged_accumulator: {merged_accumulator}")
        # merge_accumulators - merged_accumulator: (5, 23)
        return merged_accumulator

    # Wyodrębniamy końcowy wynik (różnica max - min)
    def extract_output(self, min_max):
        (min_value, max_value) = min_max
        result = max_value - min_value if min_value != float('inf') else float('NaN')
        print(f"extract_output - min_value: {min_value}, max_value: {max_value}, result: {result}")
        # extract_output - min_value: 5, max_value: 23, result: 18
        return result

# Inicjalizacja potoku
p = beam.Pipeline()

# Przykład użycia CombineGlobally
max_difference = (
    p
    | beam.Create([15, 5, 7, 7, 9, 23, 13, 5])  # Tworzenie danych wejściowych
    | "Compute Max Difference" >> beam.CombineGlobally(MaxDifferenceFn())  # Obliczanie maksymalnej różnicy
    | 'Write results' >> beam.io.WriteToText('data/max_difference')  # Zapis wyników do pliku
)

# Uruchomienie potoku
p.run()

create_accumulator: (inf, -inf)
add_input - input: 15, updated_accumulator: (15, 15)
add_input - input: 5, updated_accumulator: (5, 15)
add_input - input: 7, updated_accumulator: (5, 15)
add_input - input: 7, updated_accumulator: (5, 15)
add_input - input: 9, updated_accumulator: (5, 15)
add_input - input: 23, updated_accumulator: (5, 23)
add_input - input: 13, updated_accumulator: (5, 23)
add_input - input: 5, updated_accumulator: (5, 23)
merge_accumulators - incoming accumulators: [(5, 23)]
merge_accumulators - merged_accumulator: (5, 23)
extract_output - min_value: 5, max_value: 23, result: 18


In [12]:
!{'head -n 20 data/max_difference-00000-of-00001'}

18
